# Problème 3 : Equilibre chimique de la combustion d'hydrogène

#### numéro d'équipe : 
#### Nom, prénom et matricule des membres:
####    -
####    -
####    -

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cantera as ct
from math import*

In [ ]:
from thermoprop import *
from solveur import *
from equilibre import *   # à compléter

## 3.a Equilibre chimique

### Code principal

In [ ]:
%%time
gas = ct.Solution('SanDiego_modified.yaml')
gas.basis="molar"

# Paramètres du problème
T0 = 750                                    # Température initiale [K]
P_arr = [ct.one_atm, 10*ct.one_atm]         # Pression [Pa]
phi_arr = np.arange(0.3, 3.0001, 0.05)
species = # à remplir                       # List des espèces du problème

# liste des masses molaires des espèces pour la vitesse de flamme
Mi = [gas.molecular_weights[gas.species_index(sp)] for sp in species]
# Le cp d'un mélange peut être récupéré avec gas.cp_mole*1e-3 en [J/mol/K]

# arrays pour l'affichage
T_ad_eq = np.zeros((len(P_arr), len(phi_arr)))
x_NO_eq = np.zeros((len(P_arr), len(phi_arr)))
Sl_eq = np.zeros((len(P_arr), len(phi_arr)))
df_eq = np.zeros((len(P_arr), len(phi_arr)))

for i_P, P in enumerate(P_arr):
    # Paramètres pour le solveur
    X_eq = # à remplir                          # Estimation initiale de la composition chimique en fractions molaires
    T_est = 2400                                # Estimation initiale pour la température
    epsi_T = 1e-6                               # Tolérance relative
    max_iter = 10                               # Nombre maximal d'itération pour le solveur
    for i_phi, phi in enumerate(phi_arr):
        # Définition des réactifs
        N_R = # à remplir                       # Quantités de matière associées à chaque espèce dans les réactifs
        H_R = # à remplir                       # Enthalpie des réactifs 
        
        # Calcul de la température des produits à l'équilibre
        def fun_T(T):
            H_P = calc_H_TP_cst(T, X_eq, gas, species, P, phi)
            return H_P-H_R 
        T_eq = solveur_Newton_Raphson(fun_T, T_est, abs(H_R)*epsi_T, max_iter)
        
        # Récupération de la température adiabatique de flamme
        T_ad_eq[i_P, i_phi] = T_eq
        T_est = T_eq
        
        # Calcul de la composition des produits à la température adiabatique de flamme
        X_eq = calc_comp_TP_cst(X_eq, gas, species, T_eq, P, phi)
        x_NO_eq[i_P, i_phi] = X_eq[4]

### Affichage

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10,10), dpi=100)

for i_P, P in enumerate(P_arr):
    axs[0,0].plot(phi_arr, T_ad_eq[i_P,:], label = "P={:.0f}atm".format(P/ct.one_atm))
    axs[0,1].plot(phi_arr, x_NO_eq[i_P,:]*1e6)
    axs[1,0].plot(phi_arr, Sl_eq[i_P, :])
    axs[1,1].plot(phi_arr, df_eq[i_P, :]*1e6)

axs[0,0].set_ylabel("T [K]")
axs[0,0].set_xlabel(r"Rapport d'équivalence $\phi$")
axs[0,0].legend()

axs[0,1].set_ylabel("Concentration en NO [ppm]")
axs[0,1].set_xlabel(r"Rapport d'équivalence $\phi$")

axs[1,0].set_ylabel("Vitesse de flamme [m/s]")
axs[1,0].set_xlabel(r"Rapport d'équivalence $\phi$")

axs[1,1].set_ylabel(r"Epaisseur de flamme [$\mu m$]")
axs[1,1].set_xlabel(r"Rapport d'équivalence $\phi$")

fig.suptitle("Problème 3a\n Solveur perso")
fig.tight_layout()
plt.show()

### Comparaison avec Cantera

In [ ]:
%%time
gas = ct.Solution('SanDiego_modified.yaml')
gas.basis="molar"

# Paramètres du problème
T0 = 750                                   # Température initiale [K]
P_arr = [ct.one_atm, 10*ct.one_atm]         # Pression [Pa]

phi_arr = np.arange(0.3, 3.0001, 0.05)

T_cant = np.zeros((len(P_arr), len(phi_arr)))
x_NO_cant = np.zeros(T_cant.shape)
Sl_cant = np.zeros(T_cant.shape)
df_cant = np.zeros(T_cant.shape)
for i_P, P in enumerate(P_arr):
    for i_phi, phi in enumerate(phi_arr):
        # Calcul de l'équilibre chimique
        gas.set_equivalence_ratio(phi, fuel="H2:1", oxidizer="O2:1, N2:3.773")
        
        gas.TP = T0, P
        gas.equilibrate("HP")
        
        # Récupération de la température adiabatique de flamme et de la composition à l'équilibre chimique
        T_cant[i_P,i_phi] = gas.T
        x_NO_cant[i_P, i_phi] = gas.X[gas.species_index("NO")]
        
        # Récupération de la vitesse de flamme et de l'épaisseur de flamme calculés avec Cantera avec FreeFlame
        # (calcul trop long pour être fait ici)
        Sl_cant[0,:] = np.array([3.43050376, 4.48072621, 5.39549809, 6.24300057, 7.00650186, 7.70262113, 8.33898263, 8.92485758, 9.45401843, 9.93763617, 10.39195498, 10.79774649, 11.15708808, 11.49605519, 11.80522914, 12.08572595, 12.34100043, 12.55919662, 12.75770468, 12.94218022, 13.10611275, 13.25037362, 13.37848508, 13.48573867, 13.57632438, 13.65135447, 13.71114104, 13.75757639, 13.79127552, 13.81326326, 13.82458237, 13.82578914, 13.81885202, 13.80276600, 13.77927286, 13.74818804, 13.71273356, 13.67087927, 13.62282633, 13.57167316, 13.51661143, 13.45728229, 13.39675139, 13.33127179, 13.26426177, 13.19614674, 13.12490797, 13.05236469, 12.97883274, 12.90378050, 12.82719500, 12.75603151, 12.67703692, 12.59859959, 12.51947840])
        Sl_cant[1,:] = np.array([0.43828832, 0.89648791, 1.49065482, 2.16813182, 2.92534662, 3.70351087, 4.50879297, 5.33004685, 6.11815087, 6.87479150, 7.59270720, 8.26582009, 8.89186618, 9.46994591, 10.00432351, 10.48267526, 10.91930198, 11.30881246, 11.65563083, 11.95689368, 12.21507617, 12.43868195, 12.62810954, 12.78582385, 12.91536027, 13.01941478, 13.10001984, 13.16034105, 13.20167619, 13.22627931, 13.23803871, 13.23418827, 13.21837658, 13.19101995, 13.15483116, 13.11015997, 13.06439050, 13.00509265, 12.93871659, 12.87008365, 12.79275569, 12.71880890, 12.63793857, 12.54948633, 12.45899853, 12.36593337, 12.26889196, 12.17352430, 12.07448509, 11.97306011, 11.87131518, 11.76960067, 11.66596746, 11.55338537, 11.44259918])
        df_cant[0,:] = np.array([8.8292e-05, 7.0478e-05, 6.0846e-05, 5.4527e-05, 5.0247e-05, 4.7155e-05, 4.4825e-05, 4.3005e-05, 4.1581e-05, 4.0423e-05, 3.9407e-05, 3.8567e-05, 3.7854e-05, 3.7158e-05, 3.6493e-05, 3.5846e-05, 3.5210e-05, 3.4624e-05, 3.4055e-05, 3.3499e-05, 3.2983e-05, 3.2508e-05, 3.2073e-05, 3.1686e-05, 3.1340e-05, 3.1031e-05, 3.0756e-05, 3.0516e-05, 3.0305e-05, 3.0122e-05, 2.9966e-05, 2.9831e-05, 2.9715e-05, 2.9624e-05, 2.9547e-05, 2.9481e-05, 2.9432e-05, 2.9398e-05, 2.9370e-05, 2.9358e-05, 2.9357e-05, 2.9365e-05, 2.9374e-05, 2.9393e-05, 2.9425e-05, 2.9462e-05, 2.9488e-05, 2.9533e-05, 2.9588e-05, 2.9650e-05, 2.9714e-05, 2.9730e-05, 2.9807e-05, 2.9889e-05, 2.9972e-05])
        df_cant[1,:] = np.array([6.8761e-05, 3.5033e-05, 2.1922e-05, 1.5633e-05, 1.2002e-05, 9.7889e-06, 8.2932e-06, 7.2292e-06, 6.4734e-06, 5.9107e-06, 5.4792e-06, 5.1427e-06, 4.8745e-06, 4.6529e-06, 4.4582e-06, 4.2840e-06, 4.1212e-06, 3.9749e-06, 3.8441e-06, 3.7308e-06, 3.6337e-06, 3.5492e-06, 3.4764e-06, 3.4139e-06, 3.3607e-06, 3.3151e-06, 3.2760e-06, 3.2432e-06, 3.2155e-06, 3.1923e-06, 3.1723e-06, 3.1568e-06, 3.1445e-06, 3.1351e-06, 3.1284e-06, 3.1242e-06, 3.1201e-06, 3.1197e-06, 3.1213e-06, 3.1238e-06, 3.1291e-06, 3.1338e-06, 3.1403e-06, 3.1489e-06, 3.1587e-06, 3.1695e-06, 3.1819e-06, 3.1941e-06, 3.2079e-06, 3.2225e-06, 3.2380e-06, 3.2540e-06, 3.2711e-06, 3.2906e-06, 3.3107e-06])

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10,10), dpi=100)

for i_P, P in enumerate(P_arr):
    axs[0,0].plot(phi_arr, T_cant[i_P,:], label = "P={:.0f}atm".format(P/ct.one_atm))
    axs[0,1].plot(phi_arr, x_NO_cant[i_P,:]*1e6)
    axs[1,0].plot(phi_arr, Sl_cant[i_P, :])
    axs[1,1].plot(phi_arr, df_cant[i_P, :]*1e6)

axs[0,0].set_ylabel("T [K]")
axs[0,0].set_xlabel(r"Rapport d'équivalence $\phi$")
axs[0,0].legend()

axs[0,1].set_ylabel("Concentration en NO [ppm]")
axs[0,1].set_xlabel(r"Rapport d'équivalence $\phi$")

axs[1,0].set_ylabel("Vitesse de flamme [m/s]")
axs[1,0].set_xlabel(r"Rapport d'équivalence $\phi$")

axs[1,1].set_ylabel(r"Epaisseur de flamme [$\mu m$]")
axs[1,1].set_xlabel(r"Rapport d'équivalence $\phi$")

fig.suptitle("Problème 3a\n Cantera")
fig.tight_layout()
plt.show()

## 3.b Cinétique chimique

### Fonctions fournies pour la résolution du problème (à compléter)

In [ ]:
def calc_Kp_cin(gas, species_cin, T):
    """
    Renvoie les constantes d'équilibre Kp associées à chaque réaction calculées avec les enthalpies libres
    
    gas : objet cantera représentant le mélange étudié
    species_cin : liste des strings des espèces impliquées dans la cinétique dans le mélange
    T : température du mélange
    """
    R = ct.gas_constant*1e-3       # Constante universelle des gaz parfaits [J/mol/K]
    P0 = ct.one_atm                # Pression de référence [Pa]
    
    # Calcul des enthalpie libre de chacune des espèces dans l'équation en [J/mol]
    g = {}
    for sp in species_cin:
        g[sp] = calc_thermoprop_mix(gas, [sp], [1], T, P0)[3]
    # à remplir
    Kp = [] #à remplir
    return Kp
        
def calc_dCdt(t, C, species_cin, T):
    """
    Renvoie les dérivées des concentrations des espèces impliquées dans la cinétique par rapport au temps (taux de production)
    
    gas : objet cantera représentant le mélange étudié
    species_cin : liste des strings des espèces impliquées dans la cinétique dans le mélange
    T : température du mélange
    """
    
    # Calcul des vitesse de réaction de création des deux réactions
    # à remplir
        
    # Calcul des vitesse de réaction de destruction des deux réactions
    Kp = calc_Kp_cin(gas, species_cin,T)    # fonction à remplir
    # à remplir
    
    # Calcul des taux de production de chaque espèce
    C_sp = {}
    for i_sp, sp in enumerate(species_cin):
        C_sp[sp] = C[i_sp]
    # à remplir
    dCdt = []
    return np.array(dCdt)

### Code principal

In [ ]:
%%time
# Paramètres du problème
T0 = 750                                    # Température initiale [K]
P_arr = [ct.one_atm, 10*ct.one_atm]         # Pression [Pa]
phi_arr = [0.5, 0.6]
species = ["N2", "O2", "N", "O", "NO", "H2", "H2O", "H", "OH"]
species_cin = ["N2", "O2", "N", "O", "NO"]

# paramètres numériques
tmax = 6
dt = 1e-3
time_arr = np.arange(0, tmax+dt/2, dt)

X_NO = [[],[]]
t_NO = [[],[]]
for i_P, P in enumerate(P_arr):
    # Paramètres pour le solveur
    X_eq = # Estimation initiale de la composition chimique
    X_eq = X_eq/np.sum(X_eq)
    T_est = 2400                                                # Estimation initiale pour la température
    epsi_T = 1e-6                                               # Tolérance relative
    max_iter = 10                                               # Nombre maximal d'itération pour le solveur
    for i_phi, phi in enumerate(phi_arr):
        # Définition des réactifs
        N_R = # à remplir
        H_R = # à remplir
        
        # Calcul de la température des produits à l'équilibre
        def fun_T(T):
            H_P = calc_H_TP_cst(T, X_eq, gas, species, P, phi)
            return H_P-H_R 
        T_eq = solveur_Newton_Raphson(fun_T, T_est, abs(H_R)*epsi_T, max_iter)

        # Calcul de la composition à l'équilibre
        X_eq = calc_comp_TP_cst(X_eq, gas, species, T_eq, P, phi)
        
        # Initialisation des concentrations pour la cinétique
        C0 = # à remplir  #mol/cm^3
        
        # Résolution implicite du système d'ODE
        t = 0
        C = np.zeros((len(C0), len(time_arr)))
        C[:, 0] = C0
        for i_t, t in enumerate(time_arr[:-1]):
            dCdt = calc_dCdt(t, C[:,i_t], species_cin, T_eq) # fonction à remplir
            def fun_dCdt(C):
                return calc_dCdt(t, C, species_cin, T_eq)            
            dSdC = jacobien(fun_dCdt, dCdt, C[:,i_t])
            C[:,i_t+1] = C[:,i_t] + np.matmul(np.linalg.inv(np.eye(len(dSdC)) - dt * dSdC), dCdt) * dt;
            C0 = C[:,i_t+1]
        
        t_NO[i_P].append(time_arr)
        X_NO[i_P].append(C[4,:]*R*T_eq/P*1e6)
        
        print("Fraction molaires de NO à {:.0f} atm et phi={:.1f} : {:.0f} ppm".format(P/ct.one_atm, phi, X_NO[i_P][i_phi][-1]*1e6))

### Affichage

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10,10), dpi=100)

for i_phi, phi in enumerate(phi_arr):
    for i_P, P in enumerate(P_arr):
        axs[i_phi,0].plot(t_NO[i_P][i_phi], X_NO[i_P][i_phi]*1e6, label = "P={:.0f}atm".format(P/ct.one_atm))
        axs[i_phi,0].set_title("phi={}".format(phi))
        axs[i_phi,0].set_ylabel("Concentration en NO [ppm]")
        axs[i_phi,0].set_xlabel("t [s]")
        axs[i_phi,0].legend()
        
        axs[i_phi,1].plot(t_NO[i_P][i_phi]*1e3, X_NO[i_P][i_phi]*1e6)
        axs[i_phi,1].set_title("phi={}".format(phi))
        axs[i_phi,1].set_ylabel("Concentration en NO [ppm]")
        axs[i_phi,1].set_xlabel("t [ms]")
        axs[i_phi,1].set_xlim([0,20])

axs[0,1].set_ylim([0,50])
axs[1,1].set_ylim([0,500])

fig.suptitle("Problème 3b\n Solveur perso : Concentration de NO")
fig.tight_layout()
plt.show()

### Comparaison avec Cantera

In [ ]:
%%time
gas = ct.Solution('SanDiego_modified.yaml')
gas.basis="molar"

# Paramètres physiques
T0 = 750              # [K]
P_arr = [ct.one_atm, 10*ct.one_atm]         # [Pa]
phi_arr = [0.5, 0.6]
species = ["N2", "O2", "N", "O", "NO", "H2", "H2O", "H", "OH"]

# paramètres numériques
tmax = 6
dt = 1e-3
time_arr = np.arange(0, tmax, dt)

C = np.zeros(5)
X_NO_cant = [[],[]]
t_NO_cant = [[],[]]
for i_P, P in enumerate(P_arr):
    for i_phi, phi in enumerate(phi_arr):
        # Calcul de l'équilibre chimique
        gas.set_equivalence_ratio(phi, fuel="H2:1", oxidizer="O2:1, N2:3.773")
        gas.TP = T0, P
        gas.equilibrate("HP")
        T_eq = gas.T
        
        X_eq = np.copy(gas.X)
        
        for sp in gas.species_names:
            if sp not in species:
                X_eq[gas.species_index(sp)] = 0
        
        X_eq[gas.species_index("NO")] = 0
        X_eq[gas.species_index("N")] = 0

        X_eq = X_eq/np.sum(X_eq)
        
        gas.TPX = T_eq, P, X_eq
        
        combustor = ct.ConstPressureReactor(gas)
        combustor.volume = 1.0

        sim = ct.ReactorNet([combustor])

        t=0
        i = 0
        states = ct.SolutionArray(gas, extra=['tres'])
        states.append(combustor.thermo.state, tres=t)
        while (t < tmax):    
            i+=1
            t += dt    
            sim.advance(t)
            states.append(combustor.thermo.state, tres=t)
        
        t_NO_cant[i_P].append(states.tres)
        X_NO_cant[i_P].append(states("NO").X)

        print("Fraction molaires de NO à {:.0f} atm et phi={:.1f} : {:.0f} ppm".format(P/ct.one_atm, phi, X_NO_cant[i_P][i_phi][-1][0]*1e6))

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10,10), dpi=100)

for i_phi, phi in enumerate(phi_arr):
    for i_P, P in enumerate(P_arr):

        axs[i_phi,0].plot(t_NO_cant[i_P][i_phi], X_NO_cant[i_P][i_phi]*1e6, label = "P={:.0f}atm".format(P/ct.one_atm))
        axs[i_phi,0].set_title("phi={}".format(phi))
        axs[i_phi,0].set_ylabel("Concentration en NO [ppm]")
        axs[i_phi,0].set_xlabel("t [s]")
        axs[i_phi,0].legend()
        
        axs[i_phi,1].plot(t_NO_cant[i_P][i_phi]*1e3, X_NO_cant[i_P][i_phi]*1e6)
        axs[i_phi,1].set_title("phi={}".format(phi))
        axs[i_phi,1].set_ylabel("Concentration en NO [ppm]")
        axs[i_phi,1].set_xlabel("t [ms]")
        axs[i_phi,1].set_xlim([0,20])

axs[0,1].set_ylim([0,50])
axs[1,1].set_ylim([0,500])

fig.suptitle("Problème 3b\n Cantera : Concentration de NO")
fig.tight_layout()
plt.show()